## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,26.60,39,75,17.27,broken clouds
1,1,Kapaa,US,22.0752,-159.3190,73.99,78,75,21.85,broken clouds
2,2,Khatanga,RU,71.9667,102.5000,-2.78,98,100,6.69,overcast clouds
3,3,Jamestown,US,42.0970,-79.2353,46.99,81,90,17.18,overcast clouds
4,4,Salinopolis,BR,-0.6136,-47.3561,80.19,81,46,10.18,scattered clouds


In [57]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 0
What is the maximum temperature you would like for your trip? 100


In [58]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [59]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                135
City                   135
Country                135
Lat                    135
Lng                    135
Max Temp               135
Humidity               135
Cloudiness             135
Wind Speed             135
Current Description    135
dtype: int64

In [60]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,26.60,39,75,17.27,broken clouds
1,1,Kapaa,US,22.0752,-159.3190,73.99,78,75,21.85,broken clouds
3,3,Jamestown,US,42.0970,-79.2353,46.99,81,90,17.18,overcast clouds
4,4,Salinopolis,BR,-0.6136,-47.3561,80.19,81,46,10.18,scattered clouds
5,5,Bintulu,MY,3.1667,113.0333,77.00,100,40,4.61,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
133,133,Montes Altos,BR,-5.8333,-47.0667,91.40,52,40,5.75,scattered clouds
134,134,Edson,CA,53.5835,-116.4356,32.00,93,90,14.97,light snow
135,135,Saint-Georges,GD,12.0564,-61.7485,80.60,78,75,18.41,broken clouds
136,136,Batsfjord,NO,70.6345,29.7185,33.80,80,0,14.97,clear sky


In [69]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Yellowknife,CA,26.60,broken clouds,62.4560,-114.3525,
1,Kapaa,US,73.99,broken clouds,22.0752,-159.3190,
3,Jamestown,US,46.99,overcast clouds,42.0970,-79.2353,
4,Salinopolis,BR,80.19,scattered clouds,-0.6136,-47.3561,
5,Bintulu,MY,77.00,scattered clouds,3.1667,113.0333,
...,...,...,...,...,...,...,...
133,Montes Altos,BR,91.40,scattered clouds,-5.8333,-47.0667,
134,Edson,CA,32.00,light snow,53.5835,-116.4356,
135,Saint-Georges,GD,80.60,broken clouds,12.0564,-61.7485,
136,Batsfjord,NO,33.80,clear sky,70.6345,29.7185,


In [70]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")    

In [75]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
137,Bathsheba,BB,78.8,scattered clouds,13.2167,-59.5167,Atlantis Hotel


In [76]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [77]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [78]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))